In [1]:
import pandas as pd
import numpy as np
import json

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [9]:
#FUNCIONES

#desanida columna y devuelve un dataframe
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append(None)
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    None
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict

def desanidarVariasColumnas(columna):
    lista=[]
    for col in columna:            
        #normalizo cada columna
        colAnidada=pd.json_normalize(columna[col])
        lista.append(colAnidada)
        #meto cada columna en una lista

        #armo un df concatenando cada columna de la lista, elimino Nulos y dups
        nuevoDF=pd.concat(lista).dropna().drop_duplicates()
    return nuevoDF


In [3]:
#desanido belongs to collection
dfPeliculas = (desanidarColumna(dfMovies['belongs_to_collection']))
dfPeliculas


,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
2,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
3,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
4,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
...,...,...,...,...
3166,123720.0,Frankenstein (Hammer Series),/fKiok5TbCKWRgfNd4Rj8gPMzGJp.jpg,/gl9Gi60C9WYVkObOMll19OROhMC.jpg
3167,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
3168,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
3169,37261.0,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg


In [13]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion

dfGeneros

#no me queda claro como voy a unir cada genero con su pelicula

#considerar esto mientras sigo revisando el resto del dataframe, mas que todo cuando hago el drop dups

#REVISAR TODAS LAS COLUMNAS PORQUE TIENEN DATOS ADEMAS DE NULOS

,id,name
0,16.0,Animation
1,12.0,Adventure
2,10749.0,Romance
3,35.0,Comedy
5,28.0,Action
12,10751.0,Family
13,36.0,History
15,18.0,Drama
17,80.0,Crime
28,14.0,Fantasy


In [14]:
#desanido production_companies

dfCompanies=pd.DataFrame(dfMovies['production_companies'])
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies

,name,id
0,Pixar Animation Studios,3.0
1,TriStar Pictures,559.0
2,Warner Bros.,6194.0
3,Twentieth Century Fox Film Corporation,306.0
4,Sandollar Productions,5842.0
...,...,...
21952,SHOTS,55590.0
35711,CPH Film Fund,76915.0
3745,Lantia Cinema & Audiovisivi,53671.0
21952,Titanic s.r.o.,55591.0
